In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [3]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils as utils
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary
import time
import torch.nn.init as init

c:\Users\admin\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
"""
X_train = torch.randn([14231, 1, 9000, 18])
Y_train = torch.ones([14231, 1])
X_val = torch.randn([1581, 1, 9000, 18])
Y_val = torch.ones([1581, 1])
X_test = torch.randn([10249, 1, 9000, 18])
Y_test = torch.ones([10249, 1])
"""

'\nX_train = torch.randn([14231, 1, 9000, 18])\nY_train = torch.ones([14231, 1])\nX_val = torch.randn([1581, 1, 9000, 18])\nY_val = torch.ones([1581, 1])\nX_test = torch.randn([10249, 1, 9000, 18])\nY_test = torch.ones([10249, 1])\n'

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (18, 5))
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.padding1 = nn.ZeroPad2d(10)
        

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, (5, 5), stride=5)
        self.batchnorm2 = nn.BatchNorm2d(32, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        self.padding2 = nn.ZeroPad2d(1)

        # Layer 3
        self.conv3 = nn.Conv2d(32, 64, (5, 1), stride=5)
        self.batchnorm3 = nn.BatchNorm2d(64, False)
        self.padding3 = nn.ZeroPad2d(1)

        # Layer 4
        self.conv4 = nn.Conv2d(64, 128, (5, 1), stride=5)
        self.batchnorm4 = nn.BatchNorm2d(128, False)
        self.padding4 = nn.ZeroPad2d(1)

        # Layer 5
        self.conv5 = nn.Conv2d(128, 256, (5, 1), stride=5)
        self.batchnorm5 = nn.BatchNorm2d(256, False)
        self.padding5 = nn.ZeroPad2d(1)


        # FC
        self.fc1 = nn.Linear(5760, 1)
        init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):


        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = self.padding1(x)

        
        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = self.pooling2(x)

        

        # Layer 3
        x = F.relu(self.conv3(x))
        x = self.batchnorm3(x)
        """
        # Layer 4
        x = F.relu(self.conv4(x))
        x = self.batchnorm4(x)
        x = self.padding4(x)

        # Layer 5
        x = F.relu(self.conv5(x))
        x = self.batchnorm5(x)
        x = self.padding5(x)


        #x = x.permute(0, 3, 1, 2)

        """

        x = x.reshape(-1, x.shape[1]* x.shape[2]* x.shape[3])
        x = F.sigmoid(self.fc1(x))
        return x

if __name__ == "__main__":


    model = CNN().cuda()

    summary(model.cuda(), (1, 9000, 18))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 8983, 14]           1,456
       BatchNorm2d-2         [-1, 16, 8983, 14]              32
         ZeroPad2d-3         [-1, 16, 9003, 34]               0
            Conv2d-4          [-1, 32, 1800, 6]          12,832
       BatchNorm2d-5          [-1, 32, 1800, 6]              64
         MaxPool2d-6           [-1, 32, 450, 2]               0
            Conv2d-7            [-1, 64, 90, 1]          10,304
       BatchNorm2d-8            [-1, 64, 90, 1]             128
            Linear-9                    [-1, 1]           5,761
Total params: 30,577
Trainable params: 30,577
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.62
Forward/backward pass size (MB): 73.65
Params size (MB): 0.12
Estimated Total Size (MB): 74.39
-------------------------------------------